In [8]:
import pandas as pd
import numpy as np
import random
import datetime

https://leetcode.com/problems/sales-analysis-iii/?lang=pythondata

In [2]:
data = [[1, 'S8', 1000], [2, 'G4', 800], [3, 'iPhone', 1400]]
product = pd.DataFrame(data, columns=['product_id', 'product_name', 'unit_price']).astype({'product_id':'Int64', 'product_name':'object', 'unit_price':'Int64'})
data = [[1, 1, 1, '2019-01-21', 2, 2000], [1, 2, 2, '2019-02-17', 1, 800], [2, 2, 3, '2019-06-02', 1, 800], [3, 3, 4, '2019-05-13', 2, 2800]]
sales = pd.DataFrame(data, columns=['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']).astype({'seller_id':'Int64', 'product_id':'Int64', 'buyer_id':'Int64', 'sale_date':'datetime64[ns]', 'quantity':'Int64', 'price':'Int64'})
print(product)
print(sales)

   product_id product_name  unit_price
0           1           S8        1000
1           2           G4         800
2           3       iPhone        1400
   seller_id  product_id  buyer_id  sale_date  quantity  price
0          1           1         1 2019-01-21         2   2000
1          1           2         2 2019-02-17         1    800
2          2           2         3 2019-06-02         1    800
3          3           3         4 2019-05-13         2   2800


In [5]:
product['product_id'].unique()

<IntegerArray>
[1, 2, 3]
Length: 3, dtype: Int64

In [6]:
df = product.merge(sales, how='left', on='product_id')
df

,product_id,product_name,unit_price,seller_id,buyer_id,sale_date,quantity,price
0,1,S8,1000,1,1,2019-01-21,2,2000
1,2,G4,800,1,2,2019-02-17,1,800
2,2,G4,800,2,3,2019-06-02,1,800
3,3,iPhone,1400,3,4,2019-05-13,2,2800


In [13]:
"""
datetime64[ns]는 numpy의 datetime object이다. 파이썬의 date object와는 다른 data type이다. 둘을 비교하려면 numpy의 datetime object를 파이썬의 date object로 바꿔줘야하는데, 쉬운 방법은 다음과 같이 하는 것이다. dtype이 기존의 datetime64[ns]에서 평범한 파이썬 object로 바뀌었다.
사실 df['sale_date']가 이미 Series이고 이미 numpy datetime object이기 때문에 굳이 pd.to_datetime()를 쓰지 않아도 되었겠지만, 만약 해당 column의 값이 date의 형식은 하고 있지만 date object가 아닐 수도 있기 때문에 일단 이 함수를 썼다.
"""
df['sale_date'] = pd.to_datetime(df['sale_date']).dt.date
df['sale_date']

0    2019-01-21
1    2019-02-17
2    2019-06-02
3    2019-05-13
Name: sale_date, dtype: object

In [17]:
# datetime.date함수를 통해 원하는 날짜를 만들어줄 수 있다. datetime모듈을 import한 뒤에 사용하면 된다.
dff = df[df['sale_date']<=datetime.date(2019,3,31)]
dff

,product_id,product_name,unit_price,seller_id,buyer_id,sale_date,quantity,price
0,1,S8,1000,1,1,2019-01-21,2,2000
1,2,G4,800,1,2,2019-02-17,1,800


In [18]:
# DataFrame형태로 뽑아내는 것이다. 그런데 문제에서는 1분기에만 팔린 것만 출력하라고 했다. G4는 2분기에도 판매되었으므로 사실 출력되면 안 된다.
dff[['product_id','product_name']]

,product_id,product_name
0,1,S8
1,2,G4


pandas.Series.dt.date : https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.date.html
* pandas의 datetime은 numpy datetime64[ns] type으로 저장된다. 일반적인 python date object로 바꾸고 싶으면 이 함수를 사용하면 된다. 그럼 datetime.date(2019,1,1)로 만들었을 때와 같은 date type의 object가 반환된다. datetime이 여러 개 들어간 Series에 적용하면 python date object를 담은 numpy array를 반환한다.

In [24]:
def sales_analysis(product_df: pd.DataFrame, sales_df: pd.DataFrame) -> pd.DataFrame:

    start_date, finish_date = datetime.date(2019,1,1), datetime.date(2019,3,31)
    sales_df['sale_date'] = sales_df['sale_date'].dt.date

    first_q_sales = sales_df[
        (sales_df["sale_date"] >= start_date) & (sales_df["sale_date"] <= finish_date)
    ][["product_id"]].drop_duplicates()

    other_sales = sales_df[
        (sales_df["sale_date"] < start_date) | (sales_df["sale_date"] > finish_date)
    ][["product_id"]]

    mask = first_q_sales["product_id"].isin(other_sales["product_id"])

    selected_products = first_q_sales[~mask].merge(product_df, on="product_id")[
        ["product_id", "product_name"]
    ]

    return selected_products

In [28]:
sales.groupby('product_id')['sale_date'].agg(['min','max'])

,min,max
product_id,,
1,2019-01-21,2019-01-21
2,2019-02-17,2019-06-02
3,2019-05-13,2019-05-13


In [30]:
def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:

    sales = sales.groupby('product_id', as_index=False)['sale_date'].agg(['min','max'])
    sales = sales[
        (sales['min'].dt.date >= datetime.date(2019,1,1)) &
        (sales['max'].dt.date <= datetime.date(2019,3,31))
        ]

    return sales.merge(product, on='product_id', how='inner')[['product_id','product_name']]